# F4 Pre Trained Autoencoders

From:

- https://github.com/CompVis/latent-diffusion#pretrained-autoencoding-models




In [1]:
!ls -l

total 40
-rw-rw-r-- 1 will will  1061 May 13 18:20 LICENSE
-rw-rw-r-- 1 will will   876 Jun 20 10:28 README.md
-rw-rw-r-- 1 will will 13090 Jun 20 10:58 diffusion_f4_ae.ipynb
drwxrwxr-x 2 will will  4096 Jun 20 10:29 latent_diffusion_compvis
drwxrwxr-x 5 will will  4096 Jun 20 10:15 misc
-rw-rw-r-- 1 will will   499 Jun 20 10:58 requirements.txt
drwxrwxr-x 6 will will  4096 Jun  1 11:10 wandb


In [2]:
!pip install -r requirements.txt

  Cloning https://github.com/CompVis/latent-diffusion.git (to revision main) to /tmp/pip-install-c3tqlu25/ldm
  Running command git clone -q https://github.com/CompVis/latent-diffusion.git /tmp/pip-install-c3tqlu25/ldm
  Created wheel for latent-diffusion: filename=latent_diffusion-0.0.1-py3-none-any.whl size=1950 sha256=68b9003a993518f99a5e0cd087fa4dcd4c82f89facdbabe44ec09766a103bc89
  Stored in directory: /tmp/pip-ephem-wheel-cache-qxa3p80v/wheels/0d/95/a1/3e92a15ef32daed6dd82c8132888164a9cf1ccbf0645337894
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9682 sha256=7f2e12797e89f76f548c0938dcd34ecc93aa8da1ce5c6b82eab9ac8faffe5788
  Stored in directory: /home/will/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
  Created wheel for latent-diffusion: filename=latent_diffusion-0.0.1-py3-none-any.whl size=1950 sha256=68b9003a993518f99a5e0cd087fa4dcd4c82f89facdbabe44ec09766a103bc89
  Stored in directory: /tmp/pip-ephem-wheel-cache-qxa3p80v/whe

In [22]:
!pip install git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers

In [4]:
from latent_diffusion.models.autoencoder import AutoencoderKL

ModuleNotFoundError: No module named 'latent_diffusion'

### KL Autoencoder

In [12]:
import os
import wget
import zipfile

def download_pre_trained_ae(url, output_dir):
    if os.path.exists(output_dir):
        print("Used cache")
    else:
        filename = wget.download(url)

        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall(output_dir)

        if os.path.exists(filename):
            os.remove(filename)

ModuleNotFoundError: No module named 'wget'

In [13]:
from latent_diffusion_compvis.ldm.models.autoencoder import AutoencoderKL

ModuleNotFoundError: No module named 'latent_diffusion_compvis.ldm'

In [21]:
import clip

ModuleNotFoundError: No module named 'clip'

In [18]:
from taming-transformers.modules.vqvae.quantize import VectorQuantizer2

ModuleNotFoundError: No module named 'taming_transformers'

In [14]:
from taming.modules.vqvae.quantize import VectorQuantizer2

ModuleNotFoundError: No module named 'taming'

In [1]:
from taming.modules.vqvae.quantize import VectorQuantizer2

ImportError: cannot import name 'VectorQuantizer2' from 'taming.modules.vqvae.quantize' (c:\Users\willf\AppData\Local\Programs\Python\Python39\lib\site-packages\taming\modules\vqvae\quantize.py)